Validate modle code

In [1]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
import numpy as np


def weird_division(n, d):
    return n / d if d else 0


def validate_model(model, X, Y, fold):
    """
    validates the model with a k-fold validation which is iterated
    returns the mean accuracy, specificiy, recall, precision, f1 score and auc score
    """

    splits = 5
    iteration = 10

    acc_list = []
    specificity_list = []
    recall_list = []
    precision_list = []
    f1_list = []

    if fold == "Strat":
        folds = StratifiedKFold(n_splits=splits)
    elif fold == "K":
        folds = KFold(splits, shuffle=True)

    # Iterate "interation" times of k-fold
    for i in range(1, iteration):
        # print(f'Iteration {i}/{iteration}')

        acc_total = 0
        specificity_total = 0
        recall_total = 0
        precision_total = 0
        f1_total = 0

        for train_index, test_index in folds.split(X, Y):
            x_train = X.iloc[train_index, :]
            x_test = X.iloc[test_index, :]
            y_train = Y.iloc[train_index, :]
            y_test = Y.iloc[test_index, :]

            # scale
            sc = MinMaxScaler()
            x_train = sc.fit_transform(x_train)
            x_test = sc.transform(x_test)

            # fit model and predict
            model.fit(x_train, np.ravel(y_train))
            y_pred = model.predict(x_test)

            conf_matrix = confusion_matrix(y_test, y_pred)
            TN = conf_matrix[0][0]
            FP = conf_matrix[0][1]
            FN = conf_matrix[1][0]
            TP = conf_matrix[1][1]

            accuracy = (weird_division((TP + TN), (TP + TN + FP + FN))) * 100
            recall = weird_division(TP, (TP + FN)) * 100  # recall
            specificity = weird_division(TN, (TN + FP)) * 100
            precision = weird_division(TP, (TP + FP)) * 100
            f1_score = weird_division(2 * (recall * precision), (recall + precision))

            # sum it up
            acc_total += accuracy
            recall_total += recall
            specificity_total += specificity
            precision_total += precision
            f1_total += f1_score

        # avg
        accuracy_mean = acc_total / splits
        recall_mean = recall_total / splits
        specificity_mean = specificity_total / splits
        precision_mean = precision_total / splits
        f1_mean = f1_total / splits

        acc_list.append(accuracy_mean)
        recall_list.append(recall_mean)
        specificity_list.append(specificity_mean)
        precision_list.append(precision_mean)
        f1_list.append(f1_mean)

    # print("Accuracy for the 10 iterations: ",  acc_list) #mean accuracy acros the 6 folds for each iteration
    # print("Recall for the 10 iterations: ",  recall_list) #mean accuracy acros the 6 folds for each iteration
    # print("Specificity for the 10 iterations: ",  specificity_list)
    # print("Precision for the 10 iterations: ",  precision_list)
    # print("F1  score for the 10 iterations: ",  f1_list)

    return (
        np.mean(acc_list),
        np.mean(specificity_list),
        np.mean(recall_list),
        np.mean(precision_list),
        np.mean(f1_list),
        # np.mean(auc_list),
    )


Evaluate modle code

In [2]:
def evaluate_model(model, x_train, x_test, y_train, y_test):
    sc = MinMaxScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)

    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    conf_matrix = confusion_matrix(y_test, y_pred)
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    accuracy = ((TP + TN) / (TP + TN + FP + FN)) * 100
    recall = (TP / (TP + FN)) * 100  # recall
    specificity = (TN / (TN + FP)) * 100
    precision = (TP / (TP + FP)) * 100
    f1_score = 2 * (recall * precision) / (recall + precision)

    return accuracy, recall, specificity, precision, f1_score

Model

In [49]:
import pandas as pd
import os
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC  
from sklearn.model_selection import train_test_split
from feature_reduction import feature_reduction_lda, feature_reduction_mrmr
path = '/Users/athena.kam/Documents/Thesis/codebase/thesis-2023-athena'
os.chdir(path)

CV_SPLIT = 5

def get_best_param(x_train,y_train):
    param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
    grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=0,return_train_score=True,cv = CV_SPLIT)
    #print(grid.cv_results_)
    grid.fit(x_train,y_train)
    print(grid.best_estimator_.get_params())
    return grid

def train_test_SVC(filename:str,hold_out:bool,include_personal_q:bool, grid_search:bool,reduce:bool,model_weights:dict = {},random_state:int = 0):

    df = pd.read_csv(filename)

    if include_personal_q:
        df = df[df['noPersonalQ']!=1].reset_index(drop=True)
    else:
        df = df[df['personalQ']!=1].reset_index(drop=True)
    
    headers = df.columns
    non_embeddings_headers = []
    for header in headers:
        if header.find('embbedings')<0:
            non_embeddings_headers.append(header)
    
    X = df.drop(columns=non_embeddings_headers)
    Y = df['classification']
    
    # Feature Reduction 
    if reduce:
        #x_train = feature_reduction_pca(x_train,0.9).values
        #x_train = feature_reduction_lda(x_train,y_train)
        X = feature_reduction_mrmr(X,Y,20)
        
    else:
        x_val = X.values
        X = StandardScaler().fit_transform(x_val)
    
    # Test Train split
    if hold_out:
        x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.30,random_state=random_state)
    else:
        x_train = X
        y_train = Y


    if grid_search: 
        grid = get_best_param(x_train=x_train,y_train=y_train) 
        model_svc = grid.best_estimator_
    else:
        model_svc = SVC(C = model_weights['C'],gamma=model_weights['gamma'],kernel=model_weights['kernel'])
        
    accuracy, specificiy, recall, precision, f1_score =validate_model(model_svc,pd.DataFrame(x_train),pd.DataFrame(y_train),"Strat")
    print(f'\tAverage Accuracy: {accuracy} \n\
      Average Specificity: {specificiy} \n\
      Average Recall: {recall}\n\
      Average Precision:{precision}\n\
      Average F1 score {f1_score}\n\
      ')
    
    if hold_out:
        accuracy, specificiy, recall, precision, f1_score =evaluate_model(model_svc,x_train,x_test,y_train,y_test)
        print('____________________________________________')
        print('Evaluate model')
        print(f'\tAccuracy: {accuracy} \n\
        Specificity: {specificiy} \n\
        Recall: {recall}\n\
        Precision:{precision}\n\
        F1 score {f1_score}\n\
        ')
        return accuracy, specificiy, recall, precision, f1_score
    else: 
        return accuracy, specificiy, recall, precision, f1_score


Testing different seeds (Hold out only)

In [46]:
def train_test_SVC_TEST_avg_seeds (filename:str,hold_out:bool = True,include_personal_q:bool=False, grid_search:bool=True,reduce:bool=True,model_weights:dict = {}):
    random_states = [0,5,13,27,36,42]
    n_states = len(random_states)

    acc_list = []
    specificity_list = []
    recall_list = []
    precision_list = [] 
    f1_list = []


    for random_state in random_states:
        print(f'Random State: {random_state}')
        accuracy, specificiy, recall, precision, f1_score = train_test_SVC(filename,hold_out,include_personal_q, grid_search,reduce,model_weights,random_state)
        acc_list.append(accuracy)
        specificity_list.append(specificiy)
        recall_list.append(recall)
        precision_list.append(precision)
        f1_list.append(f1_score)

    accuracy_mean = np.mean(acc_list)
    recall_mean =  np.mean(recall_list)
    specificity_mean = np.mean(specificity_list)
    precision_mean = np.mean(precision_list)
    f1_mean = np.mean(f1_list)


    print("Accuracy list: ",  acc_list) #mean accuracy acros the 6 folds for each iteration
    print("Recall list: ",  recall_list) #mean accuracy acros the 6 folds for each iteration
    print("Specificity list: ",  specificity_list)
    print("Precision list: ",  precision_list) 
    print("F1  score list: ",  f1_list) 

    print(f'\tAverage Accuracy: {accuracy_mean} \n\
      Average Specificity: {recall_mean} \n\
      Average Recall: {specificity_mean}\n\
      Average Precision:{precision_mean}\n\
      Average F1 score {f1_mean}\n\
      ')
    


In [4]:
train_test_SVC('datasets/transformed/spontaneousDialogueOnly_google_bert_embeddings_transformed.csv',False,False,True,reduce=True)

100%|██████████| 10/10 [00:01<00:00,  8.82it/s]


{'C': 10, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 91.78571428571429 
      Average Specificity: 95.0 
      Average Recall: 86.66666666666666
      Average Precision:95.0
      Average F1 score 89.14285714285714
      


In [43]:
train_test_SVC('datasets/transformed/spontaneousDialogueOnly_google_bert_embeddings_transformed.csv', hold_out=True,include_personal_q=False,grid_search=True,reduce=True)

100%|██████████| 10/10 [00:01<00:00,  9.24it/s]


{'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'sigmoid', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 88.0 
      Average Specificity: 86.66666666666666 
      Average Recall: 80.0
      Average Precision:66.66666666666666
      Average F1 score 72.0
      
____________________________________________
Evaluate model
	Accuracy: 81.81818181818183 
        Specificity: 100.0 
        Recall: 66.66666666666666
        Precision:71.42857142857143
        F1 score 83.33333333333333
        


In [50]:
train_test_SVC_TEST_avg_seeds('datasets/transformed/spontaneousDialogueOnly_google_bert_embeddings_transformed.csv')

Random State: 0


3431.94s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3431.94s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3431.96s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3431.97s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3431.98s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3431.99s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3432.01s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3432.02s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3432.03s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3432.04s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3432.06s - pydevd: Sending message related to proc

{'C': 10, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 88.0 
      Average Specificity: 86.66666666666666 
      Average Recall: 80.0
      Average Precision:70.0
      Average F1 score 73.33333333333334
      
____________________________________________
Evaluate model
	Accuracy: 72.72727272727273 
        Specificity: 80.0 
        Recall: 66.66666666666666
        Precision:66.66666666666666
        F1 score 72.72727272727272
        
Random State: 5


100%|██████████| 20/20 [00:04<00:00,  4.19it/s]


{'C': 0.1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'poly', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 84.0 
      Average Specificity: 83.33333333333333 
      Average Recall: 83.33333333333333
      Average Precision:83.33333333333333
      Average F1 score 83.33333333333333
      
____________________________________________
Evaluate model
	Accuracy: 100.0 
        Specificity: 100.0 
        Recall: 100.0
        Precision:100.0
        F1 score 100.0
        
Random State: 13


...

{'C': 1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 76.0 
      Average Specificity: 86.66666666666666 
      Average Recall: 63.333333333333336
      Average Precision:73.33333333333333
      Average F1 score 65.33333333333334
      
____________________________________________
Evaluate model
	Accuracy: 90.9090909090909 
        Specificity: 66.66666666666666 
        Recall: 100.0
        Precision:100.0
        F1 score 80.0
        
Random State: 36


100%|██████████| 20/20 [00:04<00:00,  4.61it/s]


{'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'sigmoid', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 84.0 
      Average Specificity: 90.0 
      Average Recall: 80.0
      Average Precision:86.66666666666666
      Average F1 score 76.66666666666667
      
____________________________________________
Evaluate model
	Accuracy: 81.81818181818183 
        Specificity: 60.0 
        Recall: 100.0
        Precision:100.0
        F1 score 75.0
        
Random State: 42


100%|██████████| 20/20 [00:04<00:00,  4.47it/s]


{'C': 0.1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'poly', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 84.0 
      Average Specificity: 86.66666666666666 
      Average Recall: 80.0
      Average Precision:70.0
      Average F1 score 73.33333333333334
      
____________________________________________
Evaluate model
	Accuracy: 90.9090909090909 
        Specificity: 100.0 
        Recall: 87.5
        Precision:75.0
        F1 score 85.71428571428571
        
Accuracy list:  [72.72727272727273, 100.0, 45.45454545454545, 90.9090909090909, 81.81818181818183, 90.9090909090909]
Recall list:  [66.66666666666666, 100.0, 66.66666666666666, 100.0, 100.0, 87.5]
Specificity list:  [80.0, 100.0, 37.5, 66.66666666666666, 60.0, 100.0]
Precision list:  [66.66666666666666, 100.0, 75.0, 100.0, 100.0, 75.0]
F1  sco

In [23]:
train_test_SVC('datasets/transformed/spontaneousDialogueOnly_google_bert_embeddings_transformed.csv',False,False,True,reduce=False)


{'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.01, 'kernel': 'sigmoid', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 63.928571428571416 
      Average Specificity: 68.0 
      Average Recall: 60.0
      Average Precision:50.0
      Average F1 score 47.857142857142854
      


In [28]:
train_test_SVC('datasets/transformed/spontaneousDialogueOnly_google_gpt_embeddings_transformed.csv',False,False,True,reduce=True)

100%|██████████| 10/10 [00:02<00:00,  4.86it/s]


{'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 91.78571428571429 
      Average Specificity: 90.0 
      Average Recall: 93.33333333333333
      Average Precision:90.0
      Average F1 score 90.28571428571429
      


In [30]:
train_test_SVC('datasets/transformed/spontaneousDialogueOnly_google_gpt_embeddings_transformed.csv',True,False,True,reduce=True)

100%|██████████| 10/10 [00:01<00:00,  5.03it/s]


{'C': 0.1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 64.0 
      Average Specificity: 100.0 
      Average Recall: 0.0
      Average Precision:0.0
      Average F1 score 0.0
      
____________________________________________
Evaluate model
	Accuracy: 54.54545454545454 
        Specificity: 0.0 
        Recall: 100.0
        Precision:nan
        F1 score nan
        


In [51]:
train_test_SVC_TEST_avg_seeds('datasets/transformed/spontaneousDialogueOnly_google_gpt_embeddings_transformed.csv')

Random State: 0


100%|██████████| 20/20 [00:09<00:00,  2.13it/s]


{'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 84.0 
      Average Specificity: 100.0 
      Average Recall: 60.0
      Average Precision:60.0
      Average F1 score 60.0
      
____________________________________________
Evaluate model
	Accuracy: 90.9090909090909 
        Specificity: 80.0 
        Recall: 100.0
        Precision:100.0
        F1 score 88.88888888888889
        
Random State: 5


100%|██████████| 20/20 [00:08<00:00,  2.44it/s]


{'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 96.0 
      Average Specificity: 100.0 
      Average Recall: 93.33333333333333
      Average Precision:100.0
      Average F1 score 96.0
      
____________________________________________
Evaluate model
	Accuracy: 100.0 
        Specificity: 100.0 
        Recall: 100.0
        Precision:100.0
        F1 score 100.0
        
Random State: 13


...

{'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 92.0 
      Average Specificity: 93.33333333333333 
      Average Recall: 93.33333333333333
      Average Precision:93.33333333333333
      Average F1 score 92.0
      
____________________________________________
Evaluate model
	Accuracy: 90.9090909090909 
        Specificity: 100.0 
        Recall: 87.5
        Precision:75.0
        F1 score 85.71428571428571
        
Random State: 36


100%|██████████| 20/20 [00:08<00:00,  2.32it/s]


{'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 92.0 
      Average Specificity: 100.0 
      Average Recall: 70.0
      Average Precision:80.0
      Average F1 score 73.33333333333334
      
____________________________________________
Evaluate model
	Accuracy: 81.81818181818183 
        Specificity: 60.0 
        Recall: 100.0
        Precision:100.0
        F1 score 75.0
        
Random State: 42


100%|██████████| 20/20 [00:07<00:00,  2.61it/s]


{'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 92.0 
      Average Specificity: 93.33333333333333 
      Average Recall: 90.0
      Average Precision:93.33333333333333
      Average F1 score 89.33333333333334
      
____________________________________________
Evaluate model
	Accuracy: 100.0 
        Specificity: 100.0 
        Recall: 100.0
        Precision:100.0
        F1 score 100.0
        
Accuracy list:  [90.9090909090909, 100.0, 63.63636363636363, 90.9090909090909, 81.81818181818183, 100.0]
Recall list:  [100.0, 100.0, 100.0, 87.5, 100.0, 100.0]
Specificity list:  [80.0, 100.0, 50.0, 100.0, 60.0, 100.0]
Precision list:  [100.0, 100.0, 100.0, 75.0, 100.0, 100.0]
F1  score list:  [88.88888888888889, 100.0, 66.66666666666667, 85.714285714